In [1]:
import os
import pandas as pd
import numpy as np
import time, gc
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pretrainedmodels
from argparse import Namespace
from sklearn.utils import shuffle
from apex import amp
from sklearn.model_selection import StratifiedKFold
from efficientnet_pytorch import EfficientNet
from cvcore.data.auto_augment import RandAugment
from PIL import Image
from utils import bn_update, moving_average, copy_model


In [2]:
!ls /home/chec/data/bengali

class_map.csv		       train.csv
sample_submission.csv	       train.csv.zip
test.csv		       train_image_data_0.parquet
test_image_data_0.parquet      train_image_data_0.parquet.zip
test_image_data_0.parquet.zip  train_image_data_1.parquet
test_image_data_1.parquet      train_image_data_1.parquet.zip
test_image_data_1.parquet.zip  train_image_data_2.parquet
test_image_data_2.parquet      train_image_data_2.parquet.zip
test_image_data_2.parquet.zip  train_image_data_3.parquet
test_image_data_3.parquet      train_image_data_3.parquet.zip
test_image_data_3.parquet.zip


In [3]:
#!ls /home/chec/data/bengali

In [4]:
DATA_DIR = '/home/chec/data/bengali'

In [5]:
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
class_map_df = pd.read_csv(f'{DATA_DIR}/class_map.csv')
sample_sub_df = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

In [6]:
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [7]:
HEIGHT = 137
WIDTH = 236

In [8]:
#import albumentations as albu
def get_train_augs():
    return RandAugment(n=2, m=27)

In [9]:
#plt.imshow(x)

In [10]:
#np.arange(10).copy()

In [11]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms


class BengaliDataset(Dataset):
    def __init__(self, df, img_df, train_mode=True, test_mode=False):
        self.df = df
        self.img_df = img_df
        self.train_mode = train_mode
        self.test_mode = test_mode

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = self.get_img(row.image_id)
        orig_img = img.copy()
        #print(img.shape)
        if self.train_mode:
            augs = get_train_augs()
            #img = augs(image=img)['image']
            img = np.asarray(augs(Image.fromarray(img)))
        
        img = np.expand_dims(img, axis=-1)
        orig_img = np.expand_dims(orig_img, axis=-1)
        
        #print('###', img.shape)
        #img = np.concatenate([img, img, img], 2)
        #print('>>>', img.shape)
        
        # taken from https://www.kaggle.com/iafoss/image-preprocessing-128x128
        #MEAN = [ 0.06922848809290576,  0.06922848809290576,  0.06922848809290576]
        #STD = [ 0.20515700083327537,  0.20515700083327537,  0.20515700083327537]
        
        img = transforms.functional.to_tensor(img)
        orig_img = transforms.functional.to_tensor(orig_img)
        
        #img = transforms.functional.normalize(img, mean=MEAN, std=STD)
        
        if self.test_mode:
            return img
        elif self.train_mode:
            return img, orig_img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic, row.word_label])
        else:
            return img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic, row.word_label])
                    
    def get_img(self, img_id):
        return 255 - self.img_df.loc[img_id].values.reshape(HEIGHT, WIDTH).astype(np.uint8)

    def __len__(self):
        return len(self.df)
    
def get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=0, dev_mode=False):
    train_df = pd.read_csv(f'{DATA_DIR}/train.csv')

    train_df = shuffle(train_df, random_state=1234)

    grapheme_words = np.unique(train_df.grapheme.values)
    grapheme_words_dict = {grapheme: i for i, grapheme in enumerate(grapheme_words)}
    train_df['word_label'] = train_df['grapheme'].map(lambda x: grapheme_words_dict[x])

    print(train_df.shape)

    if dev_mode:
        img_df = pd.read_parquet(f'{DATA_DIR}/train_image_data_0.parquet').set_index('image_id')
        train_df = train_df.iloc[:1000]
    else:
        img_dfs = [pd.read_parquet(f'{DATA_DIR}/train_image_data_{i}.parquet') for i in range(4)]
        img_df = pd.concat(img_dfs, axis=0).set_index('image_id')
    print(img_df.shape)
    #split_index = int(len(train_df) * 0.9)
    
    #train = train_df.iloc[:split_index]
    #val = train_df.iloc[split_index:]
    
    kf = StratifiedKFold(5, random_state=1234, shuffle=True)
    for i, (train_idx, val_idx) in enumerate(kf.split(train_df, train_df['grapheme_root'].values)):
        if i == ifold:
            #print(val_idx)
            train = train_df.iloc[train_idx]
            val = train_df.iloc[val_idx]
            break
    assert i == ifold
    print(train.shape, val.shape)
    
    train_ds = BengaliDataset(train, img_df, True, False)
    val_ds = BengaliDataset(val, img_df, False, False)
    
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True)
    train_loader.num = len(train_ds)

    val_loader = DataLoader(val_ds, batch_size=val_batch_size, shuffle=False, num_workers=8, drop_last=False)
    val_loader.num = len(val_ds)

    return train_loader, val_loader

In [12]:
#train_loader, val_loader = get_train_val_loaders()

In [13]:
#for x in train_loader:
#    print(x)
#    break

# model

In [14]:
#import pretrainedmodels

In [15]:
#model_name = 'resnet50' # could be fbresnet152 or inceptionresnetv2
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet').cuda()
#model.eval()

In [16]:
from argparse import Namespace
import timm
from timm.models.activations import Swish, Mish
from timm.models.adaptive_avgmax_pool import SelectAdaptivePool2d

In [17]:
MEAN = [ 0.06922848809290576 ]
STD = [ 0.20515700083327537 ]

In [18]:
MEAN = [ 0.06922848809290576 ]
STD = [ 0.20515700083327537 ]
import timm
from timm.models.activations import Swish, Mish
from timm.models.adaptive_avgmax_pool import SelectAdaptivePool2d

class BengaliNet4(nn.Module):
    def __init__(self, backbone_name='se_resnext50_32x4d'):
        super(BengaliNet4, self).__init__()
        self.n_grapheme = 168
        self.n_vowel = 11
        self.n_consonant = 7
        self.n_word = 1295
        self.backbone_name = backbone_name
        
        self.num_classes = self.n_grapheme + self.n_vowel + self.n_consonant + self.n_word
        
        self.backbone = pretrainedmodels.__dict__[self.backbone_name](num_classes=1000, pretrained='imagenet')
        self.fc = nn.Linear(self.backbone.last_linear.in_features, self.num_classes)
        
        self.num_p2_features = self.backbone.layer2[-1].se_module.fc2.out_channels
        self.num_p3_features = self.backbone.layer3[-1].se_module.fc2.out_channels
        self.p2_head = nn.Conv2d(self.num_p2_features, self.num_p2_features * 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        self.p3_head = nn.Conv2d(self.num_p3_features, self.num_p3_features * 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        self.bn2 = nn.BatchNorm2d(self.num_p2_features * 4)
        self.bn3 = nn.BatchNorm2d(self.num_p3_features * 4)
        self.act2 = Swish()
        self.act3 = Swish()
        
        self.fc_aux1 = nn.Linear(self.num_p3_features * 4, self.num_classes)
        self.fc_aux2 = nn.Linear(self.num_p2_features * 4, self.num_classes)
        
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        
        for fc in [self.fc, self.fc_aux1, self.fc_aux2]:
            nn.init.zeros_(fc.bias.data)

        print('init model4')
        
    def features(self, x):
        x = self.backbone.layer0(x); #print(x.size())
        x = self.backbone.layer1(x); #print(x.size())
        x = self.backbone.layer2(x); p2 = x; p2 = self.p2_head(p2); p2 = self.bn2(p2); p2 = self.act2(p2) #print(x.size())
        x = self.backbone.layer3(x); p3 = x; p3 = self.p3_head(p3); p3 = self.bn3(p3); p3 = self.act3(p3) #print(x.size())
        x = self.backbone.layer4(x); #print(x.size())
        return x, p2, p3
        
    def logits(self, x, p2, p3):
        x = self.avg_pool(x)
        #x = F.dropout2d(x, 0.2, self.training)
        x = torch.flatten(x, 1)
        
        p2 = self.avg_pool(p2)
        p2 = torch.flatten(p2, 1)
        
        p3 = self.avg_pool(p3)
        p3 = torch.flatten(p3, 1)
        return self.fc(x), self.fc_aux1(p3), self.fc_aux2(p2)
    
    def forward(self, x):
        x = F.interpolate(x, size=(224,224), mode='bilinear', align_corners=False)
        for i in range(len(x)):
            transforms.functional.normalize(x[i], mean=MEAN, std=STD, inplace=True)
        x = torch.cat([x,x,x], 1)
        #x = self.conv0(x)
        #print(x.size())
        x, p2, p3 = self.features(x)
        x, logits_aux1, logits_aux2 = self.logits(x, p2, p3)

        return x, logits_aux1, logits_aux2

In [19]:
MODEL_DIR = './model4-ckps'
def create_model(args):
    model = BengaliNet4(args.backbone)
    model_file = os.path.join(MODEL_DIR, args.backbone, args.ckp_name)

    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)

    print('model file: {}, exist: {}'.format(model_file, os.path.exists(model_file)))

    if os.path.exists(model_file):
        print('loading {}...'.format(model_file))
        model.load_state_dict(torch.load(model_file))
    
    return model, model_file

In [20]:
#bnet = BengaliNet('se_resnext50_32x4d').cuda()

In [21]:
#bnet(torch.randn((2, 1, 137, 236)).cuda()).size()

# train

In [22]:
round(1/9, 6)

0.111111

In [23]:
import numpy as np
import sklearn.metrics
import torch


def calc_metrics(preds0, preds1, preds2, preds3, y):
    assert len(y) == len(preds0) == len(preds1) == len(preds2) == len(preds3)

    recall_grapheme = sklearn.metrics.recall_score(preds0, y[:, 0], average='macro')
    recall_vowel = sklearn.metrics.recall_score(preds1, y[:, 1], average='macro')
    recall_consonant = sklearn.metrics.recall_score(preds2, y[:, 2], average='macro')
    recall_word = sklearn.metrics.recall_score(preds3, y[:, 3], average='macro')
    
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_recall_score = np.average(scores, weights=[2, 1, 1])
    
    metrics = {}
    metrics['recall'] = round(final_recall_score, 6)
    metrics['recall_grapheme'] = round(recall_grapheme, 6)
    metrics['recall_vowel'] = round(recall_vowel, 6)
    metrics['recall_consonant'] = round(recall_consonant, 6)
    metrics['recall_word'] = round(recall_word, 6)
    
    metrics['acc_grapheme'] = round((preds0 == y[:, 0]).sum() / len(y), 6)
    metrics['acc_vowel'] = round((preds1 == y[:, 1]).sum() / len(y), 6)
    metrics['acc_consonant'] = round((preds2 == y[:, 2]).sum() / len(y), 6)
    metrics['acc_word'] = round((preds3 == y[:, 3]).sum() / len(y), 6)    
    
    return metrics

In [24]:
def criterion(outputs, y_true):
    # outputs: (N, 182)
    # y_true: (N, 3)
    
    outputs = torch.split(outputs, [168, 11, 7, 1295], dim=1)
    loss0 = F.cross_entropy(outputs[0], y_true[:, 0], reduction='mean')
    loss1 = F.cross_entropy(outputs[1], y_true[:, 1], reduction='mean')
    loss2 = F.cross_entropy(outputs[2], y_true[:, 2], reduction='mean')
    loss3 = F.cross_entropy(outputs[3], y_true[:, 3], reduction='mean')
    
    return loss0 + loss1 + loss2 + loss3 #, loss0.item(), loss1.item(), loss2.item()
    #return loss3

In [25]:
def validate(model, val_loader):
    model.eval()
    loss0, loss1, loss2, loss3 = 0., 0., 0., 0.
    preds0, preds1, preds2, preds3 = [], [], [], []
    y_true = []
    with torch.no_grad():
        for x, y in val_loader:
            y_true.append(y)
            x, y = x.cuda(), y.cuda()
            outputs, outputs_aux1, outputs_aux2 = model(x)
            #avg_outputs = torch.mean(torch.stack([outputs, outputs_aux1, outputs_aux2], 0), 0)
            outputs = torch.split(outputs, [168, 11, 7, 1295], dim=1)
            
            preds0.append(torch.max(outputs[0], dim=1)[1])
            preds1.append(torch.max(outputs[1], dim=1)[1])
            preds2.append(torch.max(outputs[2], dim=1)[1])
            preds3.append(torch.max(outputs[3], dim=1)[1])
            loss0 += F.cross_entropy(outputs[0], y[:, 0], reduction='sum').item()
            loss1 += F.cross_entropy(outputs[1], y[:, 1], reduction='sum').item()
            loss2 += F.cross_entropy(outputs[2], y[:, 2], reduction='sum').item()
            loss3 += F.cross_entropy(outputs[3], y[:, 3], reduction='sum').item()
            
            # for debug
            #metrics = {}
            #metrics['loss_grapheme'] =  F.cross_entropy(outputs[0], y[:, 0], reduction='mean').item()
            #metrics['loss_vowel'] =  F.cross_entropy(outputs[1], y[:, 1], reduction='mean').item()
            #metrics['loss_consonant'] =  F.cross_entropy(outputs[2], y[:, 2], reduction='mean').item()
            #return metrics
    
    preds0 = torch.cat(preds0, 0).cpu().numpy()
    preds1 = torch.cat(preds1, 0).cpu().numpy()
    preds2 = torch.cat(preds2, 0).cpu().numpy()
    preds3 = torch.cat(preds3, 0).cpu().numpy()
    
    y_true = torch.cat(y_true, 0).numpy()
    
    #print('y_true:', y_true.shape)
    #print('preds0:', preds0.shape)
    
    metrics = calc_metrics(preds0, preds1, preds2, preds3, y_true)
    metrics['loss_grapheme'] = round(loss0 / val_loader.num, 6)
    metrics['loss_vowel'] = round(loss1 / val_loader.num, 6)
    metrics['loss_consonant'] = round(loss2 / val_loader.num, 6)
    metrics['loss_word'] = round(loss3 / val_loader.num, 6)
    
    return metrics
            

In [26]:
def get_lrs(optimizer):
    lrs = []
    for pgs in optimizer.state_dict()['param_groups']:
        lrs.append(pgs['lr'])
    lrs = ['{:.6f}'.format(x) for x in lrs]
    return lrs

In [27]:
def save_model(model, model_file):
    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
    if isinstance(model, nn.DataParallel):
        torch.save(model.module.state_dict(), model_file)
    else:
        torch.save(model.state_dict(), model_file)

In [28]:
def mixup(data, targets, alpha=1):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)
    data = data * lam + shuffled_data * (1 - lam)
    targets = (targets, shuffled_targets, lam)

    return data, targets


def mixup_criterion(outputs, targets):
    targets1, targets2, lam = targets
    #criterion = nn.CrossEntropyLoss(reduction='mean')
    return lam * criterion(outputs, targets1) + (1 - lam) * criterion(outputs, targets2)

In [29]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [30]:
np.random.random()

0.6943253159557043

In [31]:
from over9000.over9000 import Over9000
from over9000.radam import RAdam
from gridmask import GridMask

In [32]:
from cvcore.solver import WarmupCyclicalLR
def make_optimizer(model, base_lr=4e-4, weight_decay=0., weight_decay_bias=0., epsilon=1e-3):
    """
    Create optimizer with per-layer learning rate and weight decay.
    """
    params = []
    for key, value in model.named_parameters():
        if not value.requires_grad:
            continue
        lr = base_lr
        params += [{"params": [value], "lr": lr, "weight_decay": weight_decay_bias if 'bias' in key else weight_decay}]
    
    optimizer = torch.optim.AdamW(params, lr, eps=epsilon)
    return optimizer

In [33]:
def train_epoch(args, model, train_loader, epoch, optimizer, lr_scheduler, grid):
    train_loss = 0

    for batch_idx, (img, orig_img, targets) in enumerate(train_loader):
        img, orig_img, targets  = img.cuda(), orig_img.cuda(), targets.cuda()
        batch_size = img.size(0)
        r = np.random.rand()

        if r < 0.3:
            # generate mixed sample
            lam = np.random.beta(args.beta, args.beta)
            rand_index = torch.randperm(img.size()[0]).cuda()
            target_a = targets
            target_b = targets[rand_index]
            bbx1, bby1, bbx2, bby2 = rand_bbox(img.size(), lam)
            #img[:, :, bby1:bby2, bbx1:bbx2] = img[rand_index, :, bby1:bby2, bbx1:bbx2] #for new cutmix
            img[:, :, bbx1:bbx2, bby1:bby2] = img[rand_index, :, bbx1:bbx2, bby1:bby2]
            
            # adjust lambda to exactly match pixel ratio
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))
            # compute output
            outputs, outputs_aux1, outputs_aux2 = model(img)
            loss_primary = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)
            loss_aux1 = criterion(outputs_aux1, target_a) * lam + criterion(outputs_aux1, target_b) * (1. - lam)
            loss_aux2 = criterion(outputs_aux2, target_a) * lam + criterion(outputs_aux2, target_b) * (1. - lam)
            loss = loss_primary + (loss_aux1 + loss_aux2)*0.8
        elif r > 0.7:
            img = grid(img)
            outputs, outputs_aux1, outputs_aux2 = model(img)
            loss_primary = criterion(outputs, targets)
            loss_aux1 = criterion(outputs_aux1, targets)
            loss_aux2 = criterion(outputs_aux2, targets)
            loss = loss_primary + (loss_aux1 + loss_aux2)*0.8
        else:
            orig_img, targets = mixup(orig_img, targets)
            outputs, outputs_aux1, outputs_aux2 = model(orig_img)
            loss_primary = mixup_criterion(outputs, targets)
            loss_aux1 = mixup_criterion(outputs_aux1, targets)
            loss_aux2 = mixup_criterion(outputs_aux2, targets)
            loss = loss_primary + (loss_aux1 + loss_aux2)*0.8
            #loss = criterion(outputs, targets)

        optimizer.zero_grad()
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        #loss.backward()
        lr_scheduler(optimizer, batch_idx, epoch)
        optimizer.step()            
        
        current_lr = get_lrs(optimizer)

        train_loss += loss.item()
        print('\r {:4d} | {:.6f} | {:06d}/{} | {:.4f} | {:.4f} |'.format(
            epoch, float(current_lr[0]), batch_size*(batch_idx+1), train_loader.num, 
            loss.item(), train_loss/(batch_idx+1)), end='')


In [34]:
import copy
best_metrics = 0.
best_metrics_swa = 0.


def validate_and_save(model, model_file, val_loader, save=False):
    global best_metrics
    best_key = 'recall'
    val_metrics = validate(model, val_loader)
    print('\nval:', val_metrics)
    
    if val_metrics[best_key] > best_metrics:
        best_metrics = val_metrics[best_key]
        if save:
            save_model(model, model_file)
            print('###>>>>> saved', model_file)
    model.train()

def validate_and_save_swa(model, model_file, val_loader, save=False):
    global best_metrics_swa
    best_key = 'recall'
    val_metrics = validate(model, val_loader)
    print('\nval:', val_metrics)
    
    if val_metrics[best_key] > best_metrics_swa:
        best_metrics_swa = val_metrics[best_key]
        if save:
            save_model(model, model_file)
            print('###>>>>> saved', model_file)
    model.train()


def train(args):
    model, model_file = create_model(args)
    model = model.cuda()

    swa_args = copy.deepcopy(args)
    swa_args.ckp_name = args.ckp_name + '_swa'
    swa_model, swa_model_file = create_model(swa_args)
    swa_model = swa_model.cuda()

    optimizer = make_optimizer(model)
    lr_scheduler = WarmupCyclicalLR(
        "cos", args.base_lr, args.num_epochs, iters_per_epoch=len(train_loader), warmup_epochs=args.warmup_epochs)
    
    [model, swa_model], optimizer = amp.initialize([model, swa_model], optimizer, opt_level="O1",verbosity=0)
    
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
        swa_model = nn.DataParallel(swa_model)
    
    validate_and_save(model, model_file, val_loader, save=False)

    swa_model_loaded = False
    if os.path.exists(swa_model_file):
        swa_model_loaded = True
        validate_and_save_swa(swa_model, swa_model_file, val_loader, save=False)
    
    for cycle in range(1, args.num_cycles+1):
        print('CYCLE:', cycle)
        grid = GridMask(64, 128, rotate=15, ratio=0.6, mode=1, prob=1.)

        for epoch in range(args.start_epoch, args.num_epochs):
            grid.set_prob(epoch, args.st_epochs)
            train_epoch(args, model, train_loader, epoch, optimizer, lr_scheduler, grid)
            validate_and_save(model, model_file, val_loader, save=True)
            
            if (epoch+1) == args.swa_start and cycle == 1:
                if not swa_model_loaded:
                    copy_model(swa_model, model)
                swa_n = args.swa_n
            if (epoch+1) >= args.swa_start and (epoch+1) % args.swa_freq == 0:
                print('SWA>>>:')
                moving_average(swa_model, model, 1.0 / (swa_n + 1))
                swa_n += 1
                bn_update(train_loader, swa_model)
                validate_and_save_swa(swa_model, swa_model_file, val_loader, save=True)

        #args.base_lr = 4e-4
        #args.num_epochs = 60
        #args.warmup_epochs = 3

        # reset scheduler at each cycle
        lr_scheduler = WarmupCyclicalLR(
            "cos", args.base_lr, args.num_epochs, iters_per_epoch=len(train_loader), warmup_epochs=args.warmup_epochs)

In [35]:
args = Namespace()
args.backbone = 'se_resnext50_32x4d'
args.ckp_name = 'model4_se_resnext50_fold0_224.pth'

args.base_lr = 1e-4
args.num_epochs = 60
args.start_epoch = 0
args.warmup_epochs = 5

args.num_cycles = 100
args.batch_size = 640
args.val_batch_size = 1024
args.st_epochs = 10

args.swa_start = 15
args.swa_freq = 3
args.swa_n = 10

args.beta = 1.0
args.cutmix_prob = 0.5

In [36]:
train_loader, val_loader = get_train_val_loaders(batch_size=args.batch_size, val_batch_size=args.val_batch_size, ifold=0)

(200840, 6)
(200840, 32332)
(160596, 6) (40244, 6)


In [37]:
train(args)

init model4
model file: ./model4-ckps/se_resnext50_32x4d/model4_se_resnext50_fold0_224.pth, exist: True
loading ./model4-ckps/se_resnext50_32x4d/model4_se_resnext50_fold0_224.pth...
init model4
model file: ./model4-ckps/se_resnext50_32x4d/model4_se_resnext50_fold0_224.pth_swa, exist: True
loading ./model4-ckps/se_resnext50_32x4d/model4_se_resnext50_fold0_224.pth_swa...

val: {'recall': 0.997891, 'recall_grapheme': 0.997361, 'recall_vowel': 0.998719, 'recall_consonant': 0.998121, 'recall_word': 0.996619, 'acc_grapheme': 0.996993, 'acc_vowel': 0.998882, 'acc_consonant': 0.998907, 'acc_word': 0.996695, 'loss_grapheme': 0.029212, 'loss_vowel': 0.020441, 'loss_consonant': 0.016688, 'loss_word': 0.019529}

val: {'recall': 0.998211, 'recall_grapheme': 0.99791, 'recall_vowel': 0.998928, 'recall_consonant': 0.998095, 'recall_word': 0.996816, 'acc_grapheme': 0.997292, 'acc_vowel': 0.998882, 'acc_consonant': 0.998857, 'acc_word': 0.996919, 'loss_grapheme': 0.012241, 'loss_vowel': 0.005872, 'loss_

  0%|          | 0/250 [00:00<?, ?it/s]


val: {'recall': 0.996991, 'recall_grapheme': 0.995907, 'recall_vowel': 0.998297, 'recall_consonant': 0.997853, 'recall_word': 0.995243, 'acc_grapheme': 0.995726, 'acc_vowel': 0.998435, 'acc_consonant': 0.99831, 'acc_word': 0.995328, 'loss_grapheme': 0.066748, 'loss_vowel': 0.046505, 'loss_consonant': 0.038753, 'loss_word': 0.040924}
SWA>>>:


100%|██████████| 250/250 [01:25<00:00,  3.07it/s]



val: {'recall': 0.998022, 'recall_grapheme': 0.997633, 'recall_vowel': 0.998762, 'recall_consonant': 0.998059, 'recall_word': 0.996893, 'acc_grapheme': 0.997292, 'acc_vowel': 0.998832, 'acc_consonant': 0.998807, 'acc_word': 0.996968, 'loss_grapheme': 0.012194, 'loss_vowel': 0.005771, 'loss_consonant': 0.004378, 'loss_word': 0.012638}
   15 | 0.000083 | 160000/160596 | 1.5846 | 7.4776 ||
val: {'recall': 0.99757, 'recall_grapheme': 0.99685, 'recall_vowel': 0.998725, 'recall_consonant': 0.997856, 'recall_word': 0.995902, 'acc_grapheme': 0.996496, 'acc_vowel': 0.998733, 'acc_consonant': 0.998584, 'acc_word': 0.995975, 'loss_grapheme': 0.024634, 'loss_vowel': 0.014759, 'loss_consonant': 0.013595, 'loss_word': 0.019522}
   16 | 0.000081 | 160000/160596 | 1.5796 | 8.6269 |||
val: {'recall': 0.996829, 'recall_grapheme': 0.995873, 'recall_vowel': 0.998206, 'recall_consonant': 0.997366, 'recall_word': 0.995122, 'acc_grapheme': 0.99585, 'acc_vowel': 0.998459, 'acc_consonant': 0.998285, 'acc_word

...

  0%|          | 0/250 [00:00<?, ?it/s]


val: {'recall': 0.997389, 'recall_grapheme': 0.996514, 'recall_vowel': 0.998684, 'recall_consonant': 0.997843, 'recall_word': 0.995761, 'acc_grapheme': 0.996198, 'acc_vowel': 0.998608, 'acc_consonant': 0.998484, 'acc_word': 0.995825, 'loss_grapheme': 0.034917, 'loss_vowel': 0.022724, 'loss_consonant': 0.018942, 'loss_word': 0.025357}
SWA>>>:


100%|██████████| 250/250 [01:25<00:00,  2.88it/s]



val: {'recall': 0.998227, 'recall_grapheme': 0.998016, 'recall_vowel': 0.9988, 'recall_consonant': 0.998075, 'recall_word': 0.997013, 'acc_grapheme': 0.997366, 'acc_vowel': 0.998857, 'acc_consonant': 0.998857, 'acc_word': 0.997093, 'loss_grapheme': 0.011643, 'loss_vowel': 0.005415, 'loss_consonant': 0.004617, 'loss_word': 0.012099}
   39 | 0.000025 | 160000/160596 | 12.8640 | 7.2871 ||
val: {'recall': 0.99699, 'recall_grapheme': 0.996, 'recall_vowel': 0.99814, 'recall_consonant': 0.997817, 'recall_word': 0.995066, 'acc_grapheme': 0.995701, 'acc_vowel': 0.99836, 'acc_consonant': 0.998385, 'acc_word': 0.995229, 'loss_grapheme': 0.085923, 'loss_vowel': 0.051934, 'loss_consonant': 0.044851, 'loss_word': 0.051627}
   40 | 0.000023 | 160000/160596 | 9.0140 | 6.8061 ||
val: {'recall': 0.997546, 'recall_grapheme': 0.996915, 'recall_vowel': 0.998544, 'recall_consonant': 0.99781, 'recall_word': 0.995955, 'acc_grapheme': 0.996397, 'acc_vowel': 0.998608, 'acc_consonant': 0.998509, 'acc_word': 0.9

KeyboardInterrupt: 

In [37]:
train(args)

init model4
model file: ./model4-ckps/se_resnext50_32x4d/model4_se_resnext50_fold0_224.pth, exist: True
loading ./model4-ckps/se_resnext50_32x4d/model4_se_resnext50_fold0_224.pth...
init model4
model file: ./model4-ckps/se_resnext50_32x4d/model4_se_resnext50_fold0_224.pth_swa, exist: False

val: {'recall': 0.997627, 'recall_grapheme': 0.997045, 'recall_vowel': 0.998395, 'recall_consonant': 0.998022, 'recall_word': 0.996112, 'acc_grapheme': 0.996546, 'acc_vowel': 0.998658, 'acc_consonant': 0.998584, 'acc_word': 0.996223, 'loss_grapheme': 0.076319, 'loss_vowel': 0.051092, 'loss_consonant': 0.037538, 'loss_word': 0.048984}
CYCLE: 1
    0 | 0.000020 | 160000/160596 | 12.3382 | 7.6499 ||
val: {'recall': 0.997427, 'recall_grapheme': 0.996786, 'recall_vowel': 0.998392, 'recall_consonant': 0.997744, 'recall_word': 0.995858, 'acc_grapheme': 0.996472, 'acc_vowel': 0.998608, 'acc_consonant': 0.998459, 'acc_word': 0.995925, 'loss_grapheme': 0.067789, 'loss_vowel': 0.044436, 'loss_consonant': 0.033

  0%|          | 0/250 [00:00<?, ?it/s]


val: {'recall': 0.997563, 'recall_grapheme': 0.997073, 'recall_vowel': 0.998593, 'recall_consonant': 0.997515, 'recall_word': 0.996069, 'acc_grapheme': 0.996596, 'acc_vowel': 0.998782, 'acc_consonant': 0.998608, 'acc_word': 0.996124, 'loss_grapheme': 0.043161, 'loss_vowel': 0.026244, 'loss_consonant': 0.021546, 'loss_word': 0.029395}
SWA>>>:


100%|██████████| 250/250 [01:26<00:00,  3.04it/s]



val: {'recall': 0.997889, 'recall_grapheme': 0.99737, 'recall_vowel': 0.998763, 'recall_consonant': 0.998055, 'recall_word': 0.996526, 'acc_grapheme': 0.996944, 'acc_vowel': 0.998733, 'acc_consonant': 0.998758, 'acc_word': 0.996546, 'loss_grapheme': 0.014027, 'loss_vowel': 0.006092, 'loss_consonant': 0.005158, 'loss_word': 0.014519}
###>>>>> saved ./model4-ckps/se_resnext50_32x4d/model4_se_resnext50_fold0_224.pth_swa
   15 | 0.000083 | 160000/160596 | 1.7672 | 8.0767 ||
val: {'recall': 0.997475, 'recall_grapheme': 0.997144, 'recall_vowel': 0.998114, 'recall_consonant': 0.997496, 'recall_word': 0.996385, 'acc_grapheme': 0.99677, 'acc_vowel': 0.998608, 'acc_consonant': 0.998559, 'acc_word': 0.996472, 'loss_grapheme': 0.045554, 'loss_vowel': 0.028491, 'loss_consonant': 0.024071, 'loss_word': 0.029917}
   16 | 0.000081 | 160000/160596 | 11.3628 | 8.2643 |
val: {'recall': 0.997121, 'recall_grapheme': 0.996732, 'recall_vowel': 0.997685, 'recall_consonant': 0.997332, 'recall_word': 0.995528,

...

  0%|          | 0/250 [00:00<?, ?it/s]


val: {'recall': 0.99745, 'recall_grapheme': 0.996762, 'recall_vowel': 0.998368, 'recall_consonant': 0.997908, 'recall_word': 0.995926, 'acc_grapheme': 0.996521, 'acc_vowel': 0.998584, 'acc_consonant': 0.998534, 'acc_word': 0.996049, 'loss_grapheme': 0.043414, 'loss_vowel': 0.029019, 'loss_consonant': 0.024335, 'loss_word': 0.027514}
SWA>>>:


 95%|█████████▍| 237/250 [01:22<00:04,  3.11it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



    7 | 0.000096 | 160000/160596 | 8.7048 | 7.8077 ||
val: {'recall': 0.997412, 'recall_grapheme': 0.996922, 'recall_vowel': 0.997834, 'recall_consonant': 0.997972, 'recall_word': 0.995752, 'acc_grapheme': 0.996322, 'acc_vowel': 0.99841, 'acc_consonant': 0.998435, 'acc_word': 0.99585, 'loss_grapheme': 0.057744, 'loss_vowel': 0.035491, 'loss_consonant': 0.028932, 'loss_word': 0.038661}
    8 | 0.000095 | 160000/160596 | 16.4653 | 7.0821 |
val: {'recall': 0.997278, 'recall_grapheme': 0.996564, 'recall_vowel': 0.998066, 'recall_consonant': 0.997918, 'recall_word': 0.99599, 'acc_grapheme': 0.996198, 'acc_vowel': 0.998435, 'acc_consonant': 0.998484, 'acc_word': 0.996099, 'loss_grapheme': 0.061909, 'loss_vowel': 0.041544, 'loss_consonant': 0.029071, 'loss_word': 0.042015}
    9 | 0.000093 | 160000/160596 | 1.5267 | 7.0195 |||
val: {'recall': 0.99703, 'recall_grapheme': 0.996061, 'recall_vowel': 0.998138, 'recall_consonant': 0.997861, 'recall_word': 0.995184, 'acc_grapheme': 0.996049, 'acc_vo

KeyboardInterrupt: 

In [37]:
#for param in model.backbone.parameters():
#    param.requires_grad = False

In [38]:
#validate(nn.DataParallel(model), val_loader)

In [39]:
train(args)

init model4
model file: ./model4-ckps/se_resnext50_32x4d/model4_se_resnext50_fold0_224.pth, exist: True
loading ./model4-ckps/se_resnext50_32x4d/model4_se_resnext50_fold0_224.pth...
init model4
model file: ./model4-ckps/se_resnext50_32x4d/model4_se_resnext50_fold0_224.pth, exist: True
loading ./model4-ckps/se_resnext50_32x4d/model4_se_resnext50_fold0_224.pth...

val: {'recall': 0.998106, 'recall_grapheme': 0.997877, 'recall_vowel': 0.998523, 'recall_consonant': 0.998147, 'recall_word': 0.997093, 'acc_grapheme': 0.997391, 'acc_vowel': 0.998683, 'acc_consonant': 0.998807, 'acc_word': 0.997217, 'loss_grapheme': 0.01219, 'loss_vowel': 0.005514, 'loss_consonant': 0.004544, 'loss_word': 0.012465}
CYCLE: 1
   19 | 0.000068 | 160000/160596 | 1.0700 | 8.1689 |||

  0%|          | 0/250 [00:00<?, ?it/s]


val: {'recall': 0.997691, 'recall_grapheme': 0.99708, 'recall_vowel': 0.998611, 'recall_consonant': 0.997992, 'recall_word': 0.996728, 'acc_grapheme': 0.996993, 'acc_vowel': 0.998708, 'acc_consonant': 0.998758, 'acc_word': 0.996844, 'loss_grapheme': 0.026619, 'loss_vowel': 0.018161, 'loss_consonant': 0.01498, 'loss_word': 0.018931}
SWA>>>:


100%|██████████| 250/250 [01:27<00:00,  2.98it/s]



val: {'recall': 0.99786, 'recall_grapheme': 0.997545, 'recall_vowel': 0.998446, 'recall_consonant': 0.997903, 'recall_word': 0.996715, 'acc_grapheme': 0.997018, 'acc_vowel': 0.998683, 'acc_consonant': 0.998683, 'acc_word': 0.996844, 'loss_grapheme': 0.012877, 'loss_vowel': 0.005653, 'loss_consonant': 0.004641, 'loss_word': 0.013015}
   20 | 0.000067 | 160000/160596 | 4.0042 | 7.1472 ||
val: {'recall': 0.997762, 'recall_grapheme': 0.997242, 'recall_vowel': 0.998623, 'recall_consonant': 0.997942, 'recall_word': 0.99666, 'acc_grapheme': 0.996944, 'acc_vowel': 0.998658, 'acc_consonant': 0.998608, 'acc_word': 0.996745, 'loss_grapheme': 0.028523, 'loss_vowel': 0.017524, 'loss_consonant': 0.014712, 'loss_word': 0.021564}
   21 | 0.000066 | 160000/160596 | 2.2736 | 7.6636 |||

...


val: {'recall': 0.997626, 'recall_grapheme': 0.996986, 'recall_vowel': 0.998717, 'recall_consonant': 0.997816, 'recall_word': 0.996402, 'acc_grapheme': 0.99672, 'acc_vowel': 0.998683, 'acc_consonant': 0.998584, 'acc_word': 0.996496, 'loss_grapheme': 0.027514, 'loss_vowel': 0.017973, 'loss_consonant': 0.015822, 'loss_word': 0.019701}
SWA>>>:


100%|██████████| 250/250 [01:27<00:00,  2.92it/s]



val: {'recall': 0.998084, 'recall_grapheme': 0.99773, 'recall_vowel': 0.998726, 'recall_consonant': 0.998151, 'recall_word': 0.997078, 'acc_grapheme': 0.997416, 'acc_vowel': 0.998807, 'acc_consonant': 0.998832, 'acc_word': 0.997192, 'loss_grapheme': 0.012476, 'loss_vowel': 0.00554, 'loss_consonant': 0.004656, 'loss_word': 0.0128}
   54 | 0.000018 | 160000/160596 | 1.2684 | 6.8234 ||
val: {'recall': 0.997615, 'recall_grapheme': 0.996811, 'recall_vowel': 0.998758, 'recall_consonant': 0.998079, 'recall_word': 0.996862, 'acc_grapheme': 0.996869, 'acc_vowel': 0.998782, 'acc_consonant': 0.998758, 'acc_word': 0.996968, 'loss_grapheme': 0.018192, 'loss_vowel': 0.011117, 'loss_consonant': 0.009827, 'loss_word': 0.015173}
   55 | 0.000017 | 119680/160596 | 8.8762 | 7.0344 ||

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff618a560d0>
Traceback (most recent call last):
  File "/home/chec/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/home/chec/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/home/chec/anaconda3/lib/python3.7/multiprocessing/process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "/home/chec/anaconda3/lib/python3.7/multiprocessing/popen_fork.py", line 48, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/home/chec/anaconda3/lib/python3.7/multiprocessing/popen_fork.py", line 28, in poll
    pid, sts = os.waitpid(self.pid, flag)
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
#save_model(model, model_file)

In [ ]:
del model